In [6]:
import pandas as pd
import numpy as np
df_train = pd.read_csv("mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [45]:
RNN_OUTPUT_UNITS = [64, 128]
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        layer = keras.layers.LSTM(size, return_sequences=notLast, dropout=0.2, name = 'LSTM' + str(i+1))
        current_layer = keras.layers.Bidirectional(layer, name = 'BiRNN' + str(i+1))(current_layer)
    current_layer = keras.layers.Dense(128, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(64, name='Dense2', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense3', activation='relu')(current_layer)
    logits = keras.layers.Dense(5, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [46]:

model=get_model()
model.load_weights("baseline_bidir_lstm_mitbih.h5")

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
BiRNN1 (Bidirectional)       (None, 187, 128)          33792     
_________________________________________________________________
BiRNN2 (Bidirectional)       (None, 256)               263168    
_________________________________________________________________
Dense1 (Dense)               (None, 128)               32896     
_________________________________________________________________
Dense2 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense3 (Dense)               (None, 16)                1040

In [48]:
pred_test_bidir_logits= model.predict(X_test)
pred_test_bidir= np.argmax(pred_test_bidir_logits, axis=-1)


In [49]:
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.SimpleRNN(size, return_sequences=notLast, dropout=0.2, name = 'rnn' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(5, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [50]:
model=get_model()
model.load_weights("rnn_simple_mitbih.h5")
pred_test_rnn_logits = model.predict(X_test)
pred_test_rnn= np.argmax(pred_test_rnn_logits, axis=-1)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
rnn1 (SimpleRNN)             (None, 187, 64)           4224      
_________________________________________________________________
rnn2 (SimpleRNN)             (None, 128)               24704     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                1040      
_________________________________________________________________
Output (Dense)               (None, 5)                 85  

In [51]:
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.LSTM(size, return_sequences=notLast, dropout=0.2, name = 'lstm' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(5, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model
model=get_model()
model.load_weights("lstm_simple_mitbih.h5")
pred_test_lstm_logits= model.predict(X_test)
pred_test_lstm= np.argmax(pred_test_lstm_logits, axis=-1)

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 187, 64)           16896     
_________________________________________________________________
lstm2 (LSTM)                 (None, 128)               98816     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                1040      
_________________________________________________________________
Output (Dense)               (None, 5)                 85 

In [52]:
def get_model():
    inp = Input(shape=(187, 1))
    current_layer = keras.layers.Masking(mask_value=0., input_shape=(187, 1), name='Masked')(inp)
    for i, size in enumerate(RNN_OUTPUT_UNITS):
        notLast = i + 1 < len(RNN_OUTPUT_UNITS)
        current_layer = keras.layers.GRU(size, return_sequences=notLast, dropout=0.2, name = 'GRU' + str(i+1))(current_layer)

    current_layer = keras.layers.Dense(64, name='Dense1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(16, name='Dense2', activation='relu')(current_layer)
    logits = keras.layers.Dense(5, name='Output', activation='softmax')(current_layer)
    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model
model=get_model()
file_path = "gru_simple_mitbih.h5"
model.load_weights(file_path)
pred_test_gru_logits= model.predict(X_test)
pred_test_gru= np.argmax(pred_test_gru_logits, axis=-1)

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 187, 1)            0         
_________________________________________________________________
Masked (Masking)             (None, 187, 1)            0         
_________________________________________________________________
GRU1 (GRU)                   (None, 187, 64)           12672     
_________________________________________________________________
GRU2 (GRU)                   (None, 128)               74112     
_________________________________________________________________
Dense1 (Dense)               (None, 64)                8256      
_________________________________________________________________
Dense2 (Dense)               (None, 16)                1040      
_________________________________________________________________
Output (Dense)               (None, 5)                 85 

In [53]:
import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = Convolution1D(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = Convolution1D(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.layers.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.layers.Activation("relu", name='Act' + s)(layer)
    layer = keras.layers.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def get_model():
    inp = Input(shape=(187, 1))
    current_layer = Convolution1D(filters=32, kernel_size=5, strides=1)(inp)

    for i in range(5):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.layers.Flatten()(current_layer)
    current_layer = keras.layers.Dense(128, name='FC1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(32, name='FC2', activation='relu')(current_layer)
    logits = keras.layers.Dense(5,activation=activations.softmax, name='Output')(current_layer)

    model = models.Model(inputs=inp, outputs=logits)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model
model = get_model()
file_path = "residual_cnn_mitbih.h5"
model.load_weights(file_path)
pred_test_res_cnn_logits= model.predict(X_test)
pred_test_res_cnn= np.argmax(pred_test_res_cnn_logits, axis=-1)

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 187, 1)       0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 183, 32)      192         input_12[0][0]                   
__________________________________________________________________________________________________
Conv1_1 (Conv1D)                (None, 183, 32)      5152        conv1d_2[0][0]                   
__________________________________________________________________________________________________
Conv2_1 (Conv1D)                (None, 183, 32)      5152        Conv1_1[0][0]                    
___________________________________________________________________________________________

In [87]:
#majority voting
all_votes=np.concatenate([np.reshape(pred_test_bidir,[21892,1]),np.reshape(pred_test_res_cnn,[21892,1]),np.reshape(pred_test_gru,[21892,1]),np.reshape(pred_test_lstm,[21892,1]),np.reshape(pred_test_rnn,[21892,1])],axis=1)

In [88]:
all_votes

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [4, 4, 4, 0, 0],
       [4, 4, 4, 4, 0],
       [4, 4, 4, 4, 0]])

In [89]:
#majority vote based ensembling
from scipy.stats import mode
modes=mode(all_votes,axis=1)

In [90]:
modes_only=np.array(np.squeeze(modes[0]))

In [91]:
modes_only

array([0, 0, 0, ..., 4, 4, 4])

In [92]:
f1 = f1_score(Y_test, modes_only,average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, modes_only)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.785320999000833 
Test accuracy score : 0.9572903343687191 


In [94]:
#logistic regression based ensembling
true_logits_bidir=[]
for i in range(0,np.shape(pred_test_bidir)[0]):
    true_logits_bidir.append(pred_test_bidir_logits[i,Y_test[i]])
true_logits_bidir=np.array(true_logits_bidir)
true_logits_lstm=[]
for i in range(0,np.shape(pred_test_lstm)[0]):
    true_logits_lstm.append(pred_test_lstm_logits[i,Y_test[i]])
true_logits_lstm=np.array(true_logits_lstm)
true_logits_gru=[]
for i in range(0,np.shape(pred_test_gru)[0]):
    true_logits_gru.append(pred_test_gru_logits[i,Y_test[i]])
true_logits_gru=np.array(true_logits_gru)
true_logits_rnn=[]
for i in range(0,np.shape(pred_test_rnn)[0]):
    true_logits_rnn.append(pred_test_rnn_logits[i,Y_test[i]])
true_logits_rnn=np.array(true_logits_rnn)
true_logits_res_cnn=[]
for i in range(0,np.shape(pred_test_res_cnn)[0]):
    true_logits_res_cnn.append(pred_test_res_cnn_logits[i,Y_test[i]])
true_logits_res_cnn=np.array(true_logits_res_cnn)

In [95]:
full_logits=np.concatenate([np.reshape(true_logits_bidir,[21892,1]),
                           np.reshape(true_logits_rnn,[21892,1]),
                           np.reshape(true_logits_lstm,[21892,1]),
                           np.reshape(true_logits_gru,[21892,1]),
                           np.reshape(true_logits_res_cnn,[21892,1])],axis=1)

In [96]:
full_logits_sum=np.sum(full_logits,axis=1)/6

In [97]:
from sklearn.linear_model import LogisticRegression

In [98]:
clf = LogisticRegression(random_state=0).fit(full_logits,Y_test)

/Users/rheasukthanker/opt/anaconda3/envs/renv2/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [99]:
preds=clf.predict(full_logits)

In [100]:
preds

array([0, 0, 0, ..., 2, 4, 4], dtype=int8)

In [101]:
f1 = f1_score(Y_test, preds,average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, preds)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.6012732618733752 
Test accuracy score : 0.9413484377854924 
